In [1]:
import pandas as pd
import numpy as np
test = pd.read_csv('data/test.csv')
test.head()

,index,UserId,ItemId,Click,Purchase
0,75545,BYiR2R6HXg,mRhriA8HKP,1,0
1,69937,4gqsMXs4ST,bAS10Ai3WN,1,0
2,282940,NV1iw3vjY7,UPY8VgKAN0,1,0
3,18643,u1A8zZTCQR,LEEgNGikFR,1,0
4,171888,GZKapbpZ9D,qKwZIheE8N,1,0


In [2]:
%cd /workspace/viettel-ai-challenge-track-ds/SELFRec/

/workspace/viettel-ai-challenge-track-ds/SELFRec


/opt/conda/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
grouped_items = test.groupby('UserId')['ItemId'].apply(list).to_dict()
len(grouped_items)

9618

In [63]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/training_set.csv')
predict_df = pd.read_csv('../data/public_testset.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = predict_df['user_id'].values

item_columns = predict_df.columns[1:]  # Lấy tất cả cột trừ cột uid
item_in_test_df = pd.unique(predict_df[item_columns].values.ravel())
#Lọc bỏ user_id bị lẫn trong tập test
user_list = df.UserId.unique()
item_in_test_df = list(set(item_in_test_df).difference(set(user_list)))

test_df = df[~df.ItemId.isin(item_in_test_df)].sample(frac=0.05, random_state=42)  # 90% for train

user_list = df.groupby('UserId')['ItemId'].nunique()
user_list_denoise = user_list[(user_list<=20) & (user_list>=3)].index.to_list()
final_user_list = []
final_user_list.extend(user_list_denoise)
final_user_list.extend(test_user_id)
user_list_denoise = list(set(final_user_list))

train_df = df.drop(test_df.index)
train_df = train_df[train_df.UserId.isin(user_list_denoise)] # Remaining 10% for test

train_df.shape, test_df.shape

((277638, 4), (14551, 4))

In [55]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv('../data/training_set.csv')
# predict_df = pd.read_csv('../data/test_set_private.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
# test_user_id = predict_df['user_id'].values

# user_rated = df.groupby('UserId')['ItemId'].apply(list).to_dict()

# item_columns = predict_df.columns[1:]  # Lấy tất cả cột trừ cột uid
# item_in_test_df = pd.unique(predict_df[item_columns].values.ravel())
# #Lọc bỏ user_id bị lẫn trong tập test
# user_list = df.UserId.unique()
# item_in_test_df = list(set(item_in_test_df).difference(set(user_list)))

# tmp = df.groupby('UserId')['ItemId'].size().reset_index()
# user_only_one_interact = tmp[tmp.ItemId <= 3].UserId.unique()

# test_df = df[(~df.ItemId.isin(item_in_test_df))&(~df.UserId.isin(user_only_one_interact))].sample(frac=0.05, random_state=42)  # 90% for train
# # train_df = df
# train_df = df.drop(test_df.index)  # Remaining 10% for test

# train_df.shape, test_df.shape

((375372, 4), (14551, 4))

In [64]:
test = test_df.copy()

In [83]:
user_test_id = test_df.UserId.unique()

# Get preds

In [66]:
from util.conf import ModelConf
from data.loader import FileIO
import torch
import torch.nn as nn
import torch.nn.functional as F
from base.graph_recommender import GraphRecommender
# from util.conf import OptionConf
from util.sampler import next_batch_pairwise
from base.torch_interface import TorchGraphInterface
from util.loss_torch import bpr_loss, l2_reg_loss, InfoNCE
import heapq

In [67]:
import pandas as pd
import numpy as np
import torch
item_embedding_xsim = pd.read_pickle('../data/embeddings/emb_item_XSim_denoise.pickle')
user_embedding_xsim = pd.read_pickle('../data/embeddings/emb_user_XSim_denoise.pickle')

In [ ]:
import pickle 
with open("../data/grid_search/error_user.pkl",'rb') as file:
    error_user = pickle.load(file)
error_user

In [93]:
user_test_id = [x for x in user_test_id if x not in error_user]
len(user_test_id)

8655

In [78]:
user_rated = train_df.groupby('UserId')['ItemId'].apply(list).to_dict()

import torch
user_emb = []
user2id = {}
cnt = 0
user_list_xsim = []
for user in user_embedding_xsim:
    user_emb.append(user_embedding_xsim[user])
    user_list_xsim.append(user)
    user2id[user] = cnt
    cnt+=1
user_emb = torch.tensor(user_emb)

item_emb = []
item2id = {}
id2item = {}
cnt = 0
item_list_xsim = []
for item in item_embedding_xsim:
    item_emb.append(item_embedding_xsim[item])
    item_list_xsim.append(item)
    item2id[item] = cnt
    id2item[cnt] = item
    cnt+=1
item_emb = torch.tensor(item_emb)

In [85]:
from util.algorithm import find_k_largest
error_user = []
def test():
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        print(f'\rProgress: [{"+" * ratenum}{" " * (50 - ratenum)}]{ratenum * 2}%', end='', flush=True)

    rec_list = {}
    user_count = len(user_test_id)
    for i, user in enumerate(user_test_id):
        try:
            candidates = torch.matmul(torch.tensor(user_embedding_xsim[user]).cuda(), torch.tensor(item_emb.transpose(0, 1)).cuda()).cpu().numpy()
            try: 
                rated_list = user_rated[user]
            except:
                rated_list = []
            for item in rated_list:
                try:
                    candidates[item2id[item]] = -10e8
                except:
                    pass
            ids, scores = find_k_largest(10, candidates)
            item_names = [id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
        except:
            error_user.append(user)
        if i % 100 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    return rec_list

In [86]:
rec_list = test()

Progress: [                                                  ]0%

/tmp/ipykernel_239449/3024906971.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  candidates = torch.matmul(torch.tensor(user_embedding_xsim[user]).cuda(), torch.tensor(item_emb.transpose(0, 1)).cuda()).cpu().numpy()


Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%


In [87]:
import json

with open("../data/grid_search/XSimDenoise.json", "w") as file:
    json.dump(rec_list, file)


In [90]:
with open("../data/grid_search/error_user.pkl", "wb") as file:
    pickle.dump(error_user, file)

In [101]:
with open("../data/grid_search/XSimStandard.json", "r") as file:
    rec_list = json.load(file)
    print(len(rec_list))

9618


In [102]:
data = []
for user_id in user_test_id:
    data.append([user_id] + [i[0] for i in rec_list[user_id]])
df1 = pd.DataFrame(data)
df1.to_csv('../data/grid_search/predict_XSim_standard.csv', index = False, header=False)

# Grid search

In [29]:
gt_dict = {}
gt = set(zip(test_df['UserId'], test_df['ItemId']))
for user_id, item_id in gt:
    if user_id not in gt_dict:
        gt_dict[user_id] = set()
    gt_dict[user_id].add(item_id)

In [35]:
# gt_dict['xfjV2OVaYk']

In [155]:
def calculate_ndcg_at_10(df, gt):
    ndcg_total = 0
    num_users = len(df)
    
    # Iterate through each user in the DataFrame
    for _, row in df.iterrows():
        user_id = row.iloc[0]  # user_id is in the first column
        item_ids = row[1:].values  # recommended item_ids are the remaining columns
        
        # Get the ground truth relevant items for the current user
        relevant_items = gt.get(user_id, set())
        # print(relevant_items)
        # Calculate DCG@10
        dcg = 0
        for i, item_id in enumerate(item_ids[:10]):  # Top 10 items
            # print(item_id)
            relevance = 1 if item_id in relevant_items else 0
            dcg += relevance / np.log2(i + 2)  # log2(i+2) because index is 0-based
        
        # Calculate Ideal DCG@10 (IDCG) - assume the relevant items are in the top positions
        ideal_item_ids = list(relevant_items)
        idcg = 0
        for i, item_id in enumerate(item_ids[:10]):  # Top 10 items
            relevance = 1 if i < len(ideal_item_ids) else 0  # Relevance is 1 for the top relevant items
            idcg += relevance / np.log2(i + 2)
        
        # NDCG is DCG divided by IDCG
        ndcg = dcg / idcg if idcg > 0 else 0  # Prevent division by zero
        # print(dcg, idcg)
        ndcg_total += ndcg
    # print(ndcg_total)
    # Return the average NDCG
    return ndcg_total / num_users if num_users > 0 else 0

In [33]:
import math
def NDCG(origin,res,N):
    sum_NDCG = 0
    for user in res:
        DCG = 0
        IDCG = 0
        #1 = related, 0 = unrelated
        for n, item in enumerate(res[user]):
            if item[0] in origin[user]:
                DCG+= 1.0/math.log(n+2,2)
        for n, item in enumerate(list(origin[user])[:N]):
            IDCG+=1.0/math.log(n+2,2)
        sum_NDCG += DCG / IDCG
    return round(sum_NDCG / len(res),5)

In [105]:
%cd ..

/workspace/viettel-ai-challenge-track-ds


/opt/conda/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [191]:
df1 = pd.read_csv('data/grid_search/predict_XSim_denoise.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
df2 = pd.read_csv('data/grid_search/predict_XSim_full.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
df3 = pd.read_csv('data/grid_search/predict_XSim_standard.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
df4 = pd.read_csv('runs/data_for_grid_search/DirectAU_predict.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
df5 = pd.read_csv('runs/data_for_grid_search/LightGCN_predict.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
df6 = pd.read_csv('data/grid_search/SimGCL_predict.csv', names = ['user_id', 'ItemId'])
df7 = pd.read_csv('data/grid_search/predict_RecVAE.csv', names=['user_id', 'ItemId'])


In [107]:
import ast
def create_1000(df):
    # Pad each list in the 'ItemId' column to ensure it has 1,000 items
    padded_item_ids = df["ItemId"].apply(lambda x: ast.literal_eval(x) + [0] * (1000 - len(ast.literal_eval(x))) if len(ast.literal_eval(x)) < 1000 else ast.literal_eval(x)[:1000])
    
    # Create a new DataFrame from the padded lists
    expanded_df = pd.DataFrame(
        padded_item_ids.to_list(), 
        columns=[f"item_id_{i+1}" for i in range(1000)]
    )
    
    # Add the `user_id` back to the DataFrame
    expanded_df.insert(0, "user_id", df["user_id"])
    
    return expanded_df
    
df4 = create_1000(df4)
df5 = create_1000(df5)
df6 = create_1000(df6)
df7 = create_1000(df7)

In [174]:
df3.head()

,user_id,item_id_1,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6,item_id_7,item_id_8,item_id_9,...,item_id_991,item_id_992,item_id_993,item_id_994,item_id_995,item_id_996,item_id_997,item_id_998,item_id_999,item_id_1000
0,BYiR2R6HXg,4TvzFK4vZj,5D0tcQNSPC,QsS2ZSeOHk,8IiY9QXFZd,SAT9ylfcTM,eCL613RYIJ,r4SIjbucnQ,EfrACUgZoC,8W8bbWrtd2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4gqsMXs4ST,bAS10Ai3WN,mGyqHKmCgE,3zHJo1yM3P,7Zk980ZEfp,teulFeLH2w,relvQBwWKb,NHdqdVNPAG,umAcyR4ruF,tuL0CRXFN7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NV1iw3vjY7,umFzH7COE9,mqBGE4V7X4,Ryheh0wJqW,1w2tBAZklP,1YWZ7ysFBj,LvoCoE37bj,B1RAz6j2wo,rikHZfesKt,SEruLWaiyW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,u1A8zZTCQR,2Z1kYPx6P9,T5Fl5JjMdu,MZTnWwO0KP,vQ4LdEnMYd,D9BCAKYT2X,iqis3H2tXL,gDsbAt7Hpr,akr2wUs0GU,4iIvzaIZaZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GZKapbpZ9D,OPnJ3bzTjf,g0sLjzBbdC,YhXF0wLsfa,mKg9vRmHNo,90T0MJu0DH,1mHzIaSLoz,P4qcBEBhJJ,ZX3Pbjwi0s,DFhmb9OynU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
def get_rec_list(df):
    rec_list = {}  # Initialize an empty dictionary
    for i, row in df.iterrows():
        key = row[0]
        values = row[1:11].tolist()  # Convert the remaining values to a list
        rec_list[key] = values  # Add the key-value pair to the dictionary
    return rec_list

# Example usage
rec_list = get_rec_list(df1)


/tmp/ipykernel_233847/1567586879.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  key = row[0]


In [91]:
NDCG(gt_dict, rec_list, 10)

0.17047

In [192]:
calculate_ndcg_at_10(df4, gt_dict)

np.float64(0.0008570093387721419)

In [135]:
sub1 = df1.copy()
sub2 = df2.copy()
sub3 = df3.copy()
sub4 = df4.copy()
sub5 = df5.copy()
sub = pd.DataFrame()
sub['user_id'] = sub1['user_id']
sub1['prediction0'] = sub1.apply(lambda row: ' '.join(str(row[f'item_id_{i+1}']) for i in range(1000)), axis=1)
sub2['prediction1'] = sub2.apply(lambda row: ' '.join(str(row[f'item_id_{i+1}']) for i in range(1000)), axis=1)
sub3['prediction2'] = sub3.apply(lambda row: ' '.join(str(row[f'item_id_{i+1}']) for i in range(1000)), axis=1)
# sub4['prediction3'] = sub4.apply(lambda row: ' '.join(str(row[f'item_id_{i+1}']) for i in range(1000)), axis=1)
# sub5['prediction4'] = sub5.apply(lambda row: ' '.join(str(row[f'item_id_{i+1}']) for i in range(1000)), axis=1)
sub['prediction0'] = sub1['prediction0']
sub['prediction1'] = sub2['prediction1']
sub['prediction2'] = sub3['prediction2']
# sub['prediction3'] = sub4['prediction3']
# sub['prediction4'] = sub5['prediction4']

In [132]:
# Ensemble dựa trên RRF
def cust_blend(dt, W = [1.5,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    # REC.append(dt['prediction3'].split())
    # REC.append(dt['prediction4'].split())

    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:1000])

In [142]:
def eval(sub, W):
    tmp = sub.copy()
    tmp['prediction'] = tmp.apply(lambda x: cust_blend(x, W), axis=1)
    tmp = tmp[['user_id', 'prediction']]
    value_lists = tmp['prediction'].str.split(" ")
    value_lists = value_lists.apply(lambda x: x + ['0'] * (1000 - len(x)) if len(x) < 1000 else x[:1000])
    final = pd.DataFrame(value_lists.tolist(), index=tmp['user_id']).reset_index()
    final.columns = ['user_id'] + ['item_id_' + str(i) for i in range(1, 1001)]
    return calculate_ndcg_at_10(final, gt_dict)

In [ ]:
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.001
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

In [ ]:
def grid(*args):
    if len(args) == 1:
        for k in args[0]:
            yield [k]
    else:
        for k in args[0]:
            for rest in grid(*args[1:]):
                yield([k] + rest)

In [161]:
from tqdm import tqdm
f = open('data/grid_search/grid_search.txt', 'w')
res = []
best = 0
best_W = []

range_outer = range(1, 21, 1)
range_middle = range(1, 21, 1)
range_inner = range(1, 21, 1)
total_iterations = len(range_outer) * len(range_middle) * len(range_inner)
count = 0
with tqdm(total=total_iterations, desc="Total Progress") as pbar:
    for XSimstd in range_outer:
        for XSimfull in range_middle:
            for XSimDenoise in range_inner:
                pbar.update(1)
                count += 1
                stop = 0
                W = [XSimstd/10, XSimfull/10, XSimDenoise/10]
                ndcg = eval(sub, W)
                res.append([ndcg] + W)
                if ndcg<=best:
                    stop += 1
                else:
                    stop = 0
                    best = ndcg
                    best_W = W
                    f.write(f'{W}\n')
                if count%100 == 0:
                    print('Current Best W: ', best_W)
                    print('Current Best ndcg: ', best)
                # if stop >= 10: break
sorted_list = sorted(res, key=lambda x: x[0], reverse=True)


Total Progress:   1%|▏         | 101/8000 [09:33<12:35:56,  5.74s/it]

Current Best W:  [0.1, 0.4, 0.5]
Current Best ndcg:  0.13266021310281484


Total Progress:   3%|▎         | 201/8000 [19:11<12:29:49,  5.77s/it]

Current Best W:  [0.1, 0.4, 0.5]
Current Best ndcg:  0.13266021310281484


Total Progress:   4%|▍         | 301/8000 [28:48<12:13:52,  5.72s/it]

Current Best W:  [0.1, 1.3, 1.4]
Current Best ndcg:  0.13275347559009176


Total Progress:   5%|▍         | 397/8000 [38:07<12:10:14,  5.76s/it]


KeyboardInterrupt: 

In [ ]:
with open('data/grid_search/grid_search.txt', 'w') as f:
    for line in sorted_list[0]:
        f.write(f"{line}\n")